In [288]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [215]:
link="https://en.wikipedia.org/wiki/List_of_American_films_of_2018"

In [216]:
df1 = pd.read_html(link, header=0)[2]
df2=pd.read_html(link, header=0)[3]
df3=pd.read_html(link,header=0)[4]
df4=pd.read_html(link,header=0)[5]

In [217]:
df = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)

In [218]:
from tmdbv3api import TMDb
from tmdbv3api import Movie
import requests
import json
tmdb=TMDb()
tmdb_movie=Movie()
tmdb.api_key="3cd610e5da6dcc2d11cb264184e57edb"

In [219]:
from tmdbv3api import Movie
tmdb_movie = Movie()
def get_genre(x):
    genres = []
    result = tmdb_movie.search(x)
    movie_id = result[0].id
    response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id,tmdb.api_key))
    data_json = response.json()
    if data_json['genres']:
        genre_str = " " 
        for i in range(0,len(data_json['genres'])):
            genres.append(data_json['genres'][i]['name'])
        return genre_str.join(genres)
    else:
        pd.NA

In [220]:
df.head(3)

,Opening,Opening.1,Title,Production company,Cast and crew,Ref.
0,JANUARY,5,Insidious: The Last Key,Universal Pictures / Blumhouse Productions / S...,Adam Robitel (director); Leigh Whannell (scree...,[2]
1,JANUARY,5,The Strange Ones,Vertical Entertainment,Lauren Wolkstein (director); Christopher Radcl...,[3]
2,JANUARY,5,Stratton,Momentum Pictures,"Simon West (director); Duncan Falconer, Warren...",[4]


In [221]:
df.dtypes

Opening               object
Opening.1              int64
Title                 object
Production company    object
Cast and crew         object
Ref.                  object
dtype: object

In [222]:
df["genres"]=df["Title"].map(lambda x: get_genre(str(x)))

In [223]:
df_2018=df[["Title","Cast and crew","genres"]]

In [224]:
df_2018.head(2)

,Title,Cast and crew,genres
0,Insidious: The Last Key,Adam Robitel (director); Leigh Whannell (scree...,Horror Mystery Thriller
1,The Strange Ones,Lauren Wolkstein (director); Christopher Radcl...,Thriller Drama


In [225]:
df_2018["Cast and crew"][230]

'Matthew Heineman (director); Arash Amel (screenplay); Rosamund Pike, Jamie Dornan, Stanley Tucci, Tom Hollander'

In [226]:
def get_director(x):
    if "(director)" in x:
        return x.split("(director)")[0]
    elif "(director/screenplay)" in x:
        return x.split("(director/screenplay)")[0]
    else:
        return x.split("(directors)")[0]
    
    

In [251]:
df_2018["director_name"]=df_2018["Cast and crew"].map(lambda x:get_director(x))

In [237]:
df_2018["Cast and crew"][2].split("(screenplay);")[-1].split(",")[0]

' Dominic Cooper'

In [241]:
df_2018["Cast and crew"][100]

'Ben Falcone (director/screenplay); Melissa McCarthy (screenplay); Melissa McCarthy, Maya Rudolph, Molly Gordon, Julie Bowen, Gillian Jacobs, Debby Ryan, Matt Walsh, Jacki Weaver'

In [253]:
def get_actor1(x):
    return ((x.split("screenplay); ")[-1]).split(", ")[0])

In [254]:
df_2018["actor_1_name"]=df_2018["Cast and crew"].map(lambda x: get_actor_1(x))

In [255]:
def get_actor_2(x):
    if len(x.split("(screenplay);")[-1].split(","))<=1:
        return np.NaN
    else:
        return x.split("(screenplay);")[-1].split(",")[1]

In [256]:
df_2018["actor_2_name"]=df_2018["Cast and crew"].map(lambda x: get_actor_2(x))

In [257]:
def get_actor_3(x):
    if len(x.split("(screenplay);")[-1].split(","))<=2:
        return np.NaN
    else:
        return x.split("(screenplay);")[-1].split(",")[2]

In [258]:
df_2018["actor_3_name"]=df_2018["Cast and crew"].map(lambda x: get_actor_3(x))

In [259]:
df_2018.head(2)

,Title,Cast and crew,genres,director_name,actor_1_name,actor_2_name,actor_3_name
0,Insidious: The Last Key,Adam Robitel (director); Leigh Whannell (scree...,Horror Mystery Thriller,Adam Robitel,Lin Shaye,Angus Sampson,Leigh Whannell
1,The Strange Ones,Lauren Wolkstein (director); Christopher Radcl...,Thriller Drama,Lauren Wolkstein,Lauren Wolkstein (director); Christopher Radcl...,James Freedson-Jackson,Emily Althaus


In [261]:
df_2018=df_2018.rename(columns={"Title":"movie_title"})

In [262]:
newdf_2018=df_2018[["director_name","actor_1_name","actor_2_name","actor_3_name","genres","movie_title"]]

In [263]:
newdf_2018.head(2)

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title
0,Adam Robitel,Lin Shaye,Angus Sampson,Leigh Whannell,Horror Mystery Thriller,Insidious: The Last Key
1,Lauren Wolkstein,Lauren Wolkstein (director); Christopher Radcl...,James Freedson-Jackson,Emily Althaus,Thriller Drama,The Strange Ones


In [277]:
newdf_2018.isna().sum()

director_name     0
actor_1_name      0
actor_2_name      0
actor_3_name     25
genres            3
movie_title       0
dtype: int64

In [291]:
newdf_2018["actor_2_name"]=newdf_2018["actor_2_name"].replace(pd.NA,"unknown")
newdf_2018["actor_3_name"]=newdf_2018["actor_3_name"].replace(pd.NA,"unknown")
newdf_2018["genres"]=newdf_2018["genres"].replace(pd.NA,"unknown")

In [292]:
newdf_2018.isna().sum()

director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
genres           0
movie_title      0
dtype: int64

In [293]:
newdf_2018["comb"]=newdf_2018["actor_1_name"]+" "+newdf_2018["actor_2_name"]+" "+newdf_2018["actor_3_name"]+" "+\
                   newdf_2018["director_name"]+" "+newdf_2018["genres"]

In [312]:
newdf_2018["movie_title"]=newdf_2018["movie_title"].str.lower()

In [313]:
newdf_2018.head(2)

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,Adam Robitel,Lin Shaye,Angus Sampson,Leigh Whannell,Horror Mystery Thriller,insidious: the last key,Lin Shaye Angus Sampson Leigh Whannell Adam...
1,Lauren Wolkstein,Lauren Wolkstein (director); Christopher Radcl...,James Freedson-Jackson,Emily Althaus,Thriller Drama,the strange ones,Lauren Wolkstein (director); Christopher Radcl...


## Extracting features of 2019 movies from wikipedia

In [297]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2019"
df1=pd.read_html(link,header=0)[2]
df2=pd.read_html(link,header=0)[3]
df3=pd.read_html(link,header=0)[4]
df4=pd.read_html(link,header=0)[5]

In [298]:
df=df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)

In [299]:
df.shape

(242, 6)

In [300]:
df.head(3)

,Opening,Opening.1,Title,Production company,Cast and crew,Ref.
0,JANUARY,4,Escape Room,Columbia Pictures,"Adam Robitel (director); Bragi F. Schut, Maria...",[2]
1,JANUARY,4,Rust Creek,IFC Films,Jen McGowan (director); Julie Lipson (screenpl...,[3]
2,JANUARY,4,American Hangman,Hangman Justice Productions,Wilson Coneybeare (director/screenplay); Donal...,[4]


In [301]:
df["genres"]=df["Title"].map(lambda x:get_genre(str(x)))

In [302]:
df_2019=df[["Title","genres","Cast and crew"]]

In [303]:
df_2019.head()

,Title,genres,Cast and crew
0,Escape Room,Thriller Action Mystery Adventure Horror,"Adam Robitel (director); Bragi F. Schut, Maria..."
1,Rust Creek,Thriller Drama,Jen McGowan (director); Julie Lipson (screenpl...
2,American Hangman,Thriller,Wilson Coneybeare (director/screenplay); Donal...
3,A Dog's Way Home,Drama Adventure Family,Charles Martin Smith (director); W. Bruce Came...
4,The Upside,Comedy Drama,Neil Burger (director); Jon Hartmere (screenpl...


In [306]:
df_2019["Cast and crew"][30]

'J. C. Chandor (director/screenplay); Mark Boal (screenplay); Ben Affleck, Oscar Isaac, Charlie Hunnam, Garrett Hedlund, Pedro Pascal'

In [307]:
def get_director(x):
    if "(director)" in x:
        return x.split("(director)")[0]
    elif "(directors)" in x:
        return x.split("(directors)")[0]
    else:
        return x.split("(director/screenplay)")[0]
    

In [308]:
df_2019["director_name"]=df_2019["Cast and crew"].map(lambda x:get_director(x))

In [311]:
df_2019["actor_1_name"]=df_2019["Cast and crew"].map(lambda x: get_actor_1(x))
df_2019["actor_2_name"]=df_2019["Cast and crew"].map(lambda x: get_actor_2(x))
df_2019["actor_3_name"]=df_2019["Cast and crew"].map(lambda x: get_actor_3(x))

In [317]:
df_2019=df_2019.rename(columns={"Title":"movie_title"})

In [318]:
df_2019.head()

,movie_title,genres,Cast and crew,director_name,actor_1_name,actor_2_name,actor_3_name
0,Escape Room,Thriller Action Mystery Adventure Horror,"Adam Robitel (director); Bragi F. Schut, Maria...",Adam Robitel,Taylor Russell,Logan Miller,Deborah Ann Woll
1,Rust Creek,Thriller Drama,Jen McGowan (director); Julie Lipson (screenpl...,Jen McGowan,Hermione Corfield,Jay Paulson,Sean O'Bryan
2,American Hangman,Thriller,Wilson Coneybeare (director/screenplay); Donal...,Wilson Coneybeare,Wilson Coneybeare (director/screenplay); Donal...,Vincent Kartheiser,Oliver Dennis
3,A Dog's Way Home,Drama Adventure Family,Charles Martin Smith (director); W. Bruce Came...,Charles Martin Smith,Bryce Dallas Howard,Edward James Olmos,Alexandra Shipp
4,The Upside,Comedy Drama,Neil Burger (director); Jon Hartmere (screenpl...,Neil Burger,Bryan Cranston,Kevin Hart,Nicole Kidman


In [319]:
df_2019["movie_title"]=df_2019["movie_title"].str.lower()

In [320]:
newdf_2019=df_2019.loc[:,["director_name","actor_1_name","actor_2_name","actor_3_name","genres","movie_title"]]

In [322]:
newdf_2019.isna().sum()

director_name     0
actor_1_name      0
actor_2_name      1
actor_3_name     15
genres            0
movie_title       0
dtype: int64

In [323]:
newdf_2019["actor_2_name"]=newdf_2019["actor_2_name"].replace(pd.NA,"unknown")
newdf_2019["actor_3_name"]=newdf_2019["actor_3_name"].replace(pd.NA,"unknown")

In [324]:
newdf_2019.isna().sum()

director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
genres           0
movie_title      0
dtype: int64

In [325]:
newdf_2019["comb"]=newdf_2019["actor_1_name"]+" "+newdf_2019["actor_2_name"]+" "+newdf_2019["actor_3_name"]+" "+\
                   newdf_2019["director_name"]+" "+newdf_2019["genres"]

In [326]:
my_df=newdf_2018.append(newdf_2019,ignore_index=True)

In [328]:
my_df.head()

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,Adam Robitel,Lin Shaye,Angus Sampson,Leigh Whannell,Horror Mystery Thriller,insidious: the last key,Lin Shaye Angus Sampson Leigh Whannell Adam...
1,Lauren Wolkstein,Lauren Wolkstein (director); Christopher Radcl...,James Freedson-Jackson,Emily Althaus,Thriller Drama,the strange ones,Lauren Wolkstein (director); Christopher Radcl...
2,Simon West,Dominic Cooper,Austin Stowell,Gemma Chan,Action Thriller,stratton,Dominic Cooper Austin Stowell Gemma Chan Si...
3,Warwick Thornton,Bryan Brown,Sam Neill,unknown,Drama History Western,sweet country,Bryan Brown Sam Neill unknown Warwick Thornt...
4,Jaume Collet-Serra,Liam Neeson,Vera Farmiga,Patrick Wilson,Action Thriller,the commuter,Liam Neeson Vera Farmiga Patrick Wilson Jau...


In [329]:
old_df=pd.read_csv("new_data.csv")

In [330]:
old_df.shape

(5435, 7)

In [331]:
old_df.isna().sum()

director_name     3
actor_1_name     21
actor_2_name     53
actor_3_name     68
genres            6
movie_title       0
comb             71
dtype: int64

In [333]:
old_df.dropna(inplace=True)

In [334]:
final_df=old_df.append(my_df,ignore_index=True)

In [335]:
final_df.head(3)

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,Action Adventure Fantasy Sci-Fi,avatar,CCH Pounder Joel David Moore Wes Studi James C...
1,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Action Adventure Fantasy,pirates of the caribbean: at world's end,Johnny Depp Orlando Bloom Jack Davenport Gore ...
2,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Action Adventure Thriller,spectre,Christoph Waltz Rory Kinnear Stephanie Sigman ...


In [338]:
final_df.dropna(inplace=True)

In [ ]:
final_df